## Reading data from a stream

In [1]:
import earthkit.data as ekd

In [2]:
ekd.download_example_file("test6.grib")

### Getting single items from the stream

We create a stream from a file containing 6 GRIB fields by simply calling *open()*. It returns an io.BufferedReader object (a file stream).

In [3]:
stream = open("test6.grib", "rb")

In [4]:
ds = ekd.from_source("stream", stream)

In [5]:
for f in ds:
    # f is GribField object. It gets deleted when going out of scope
    print(f)

GribField(t,1000,20180801,1200,0,0)
GribField(u,1000,20180801,1200,0,0)
GribField(v,1000,20180801,1200,0,0)
GribField(t,850,20180801,1200,0,0)
GribField(u,850,20180801,1200,0,0)
GribField(v,850,20180801,1200,0,0)


Having finished the iteration there is no data available in *ds*.  We can close the stream:

In [6]:
stream.close()

### Using batched

In [7]:
stream = open("test6.grib", "rb")
ds = ekd.from_source("stream", stream)

# f is a fieldlist
for f in ds.batched(2):
    print(f"len={len(f)} {f.metadata(('param', 'level'))}")

len=2 [('t', 1000), ('u', 1000)]
len=2 [('v', 1000), ('t', 850)]
len=2 [('u', 850), ('v', 850)]


Having finished the iteration there is no data available in *ds*.  We can close the stream:

In [8]:
stream.close()

In [9]:
stream = open("test6.grib", "rb")
ds = ekd.from_source("stream", stream)

# f is a fieldlist
for f in ds.batched(4):
    print(f"len={len(f)} {f.metadata(('param', 'level'))}")

len=4 [('t', 1000), ('u', 1000), ('v', 1000), ('t', 850)]
len=2 [('u', 850), ('v', 850)]


### Using group_by

In [10]:
stream = open("test6.grib", "rb")
ds = ekd.from_source("stream", stream)

# f is a fieldlist
for f in ds.group_by("level"):
    print(f"len={len(f)} {f.metadata(('param', 'level'))}")

len=3 [('t', 1000), ('u', 1000), ('v', 1000)]
len=3 [('t', 850), ('u', 850), ('v', 850)]


Having finished the iteration there is no data available in *ds*.  We can close the stream:

In [11]:
stream.close()

### Storing each GRIB message in memory

In [12]:
stream = open("test6.grib", "rb")
ds = ekd.from_source("stream", stream, read_all=True)

In [13]:
len(ds)

6

In [14]:
ds.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
2,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
3,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
4,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
5,ecmf,v,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [15]:
a = ds.sel(param="t")
a.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [16]:
a = a.to_xarray()
a

<xarray.Dataset> Size: 2kB
Dimensions:    (levelist: 2, latitude: 7, longitude: 12)
Coordinates:
  * levelist   (levelist) int64 16B 850 1000
  * latitude   (latitude) float64 56B 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude  (longitude) float64 96B 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
Data variables:
    t          (levelist, latitude, longitude) float64 1kB ...
Attributes: (12/13)
    param:        t
    paramId:      130
    class:        od
    stream:       oper
    levtype:      pl
    type:         an
    ...           ...
    date:         20180801
    time:         1200
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

We close the stream:

In [17]:
stream.close()